In [0]:
# built-in libs
# !pip install whoosh
# !pip install pytrec_eval
import json, re 
# data analysis libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


from whoosh import index, writing
from whoosh.fields import Schema, TEXT, KEYWORD, ID, STORED
from whoosh.analysis import *
from whoosh.qparser import QueryParser
from whoosh import qparser#used in Q4
from whoosh import scoring#used in Q4
import os, os.path
from pathlib import Path
import tempfile
import subprocess
import pytrec_eval
import nltk
from nltk.stem import *
from whoosh.analysis import Filter
from whoosh import qparser

# from difflib import SequenceMatcher

## Introduction

In this part 4, we would use build a recommender system to help suer/student to select courses based on thier demands. The demand could come from the job requirement, and their own interest. Students may come from different department who are interest on data science and excited to find a career related to data science. They may not have suffcient technical skill, and management skill to fit the requirement of data scientist jobs. Therefore, this recommender system bring a insight of helping student make up their missing skills, and gain competitiveness in data science field.  

For example, if a student want to find a job name data engieer. The job require python, data mining skill,visualization data, sql and hadoop. Then, this system need student to consider what skills they lack and what skills they want to improve. Next, fill the questionnaire for the recommender system to get the recommending courses. 



## Upload online course/section description, elective couse and UofT course

Our database are obtained from the most popular online course, the elective courses from the UofT Emphasis in Analytics, and courses from our designed program.

In [0]:
# uoft course
uoft_data=pd.read_csv('Course_Analytics_Emphasis.csv',index_col=0)
# course designed by our group
own_data=pd.read_csv('Designed_course.csv',index_col=0)

own_data=own_data.reset_index()
uoft_data=uoft_data.dropna()
uoft_data=uoft_data.reset_index()

In [0]:
# business course 
university_bs_data=pd.read_csv("university_courses.csv",index_col=0)
bs_data=pd.read_csv("chapter_topics_ba.csv",index_col=0)

#technical courses
coursera_tech_data=pd.read_csv("chapter_topics.csv",index_col=0)
datacamp_data = pd.read_csv("datacamp.csv",index_col=0)
#
sql_data=pd.read_csv("chapter_topics_sql.csv",index_col=0)
big_data=pd.read_csv('chapter_topics_bigdata.csv',index_col=0)
java_data=pd.read_csv("chapter_topics_java.csv",index_col=0)
la_data=pd.read_csv('chapter_topics_la.csv',index_col=0)
nlp_data=pd.read_csv("chapter_topics_nlp.csv",index_col=0)
python_data=pd.read_csv("chapter_topics_python.csv",index_col=0)
scala_data=pd.read_csv("chapter_topics_scala.csv",index_col=0)
stats_data=pd.read_csv("chapter_topics_stats.csv",index_col=0)

#AI
AI_data=pd.read_csv("chapter_topics_ai.csv")



In [0]:
df_all_course=pd.concat([university_bs_data[['course','courses_description']].rename(columns={"course": "Course Names", "courses_description": "Course Descriptions"}),\
                        bs_data[['Course Names','Course Descriptions',"Chapter Links"]],\
                        coursera_tech_data[['Course Names','Course Descriptions',"Chapter Links"]],\
                        datacamp_data[['Course Names','Course Descriptions']],\
                        sql_data[['Course Names','Course Descriptions',"Chapter Links"]],\
                        big_data[['Course Names','Course Descriptions',"Chapter Links"]],\
                        java_data[['Course Names','Course Descriptions',"Chapter Links"]],\
                        la_data[['Course Names','Course Descriptions',"Chapter Links"]],\
                        nlp_data[['Course Names','Course Descriptions',"Chapter Links"]],\
                        python_data[['Course Names','Course Descriptions',"Chapter Links"]],\
                        scala_data[['Course Names','Course Descriptions',"Chapter Links"]],\
                        stats_data[['Course Names','Course Descriptions',"Chapter Links"]],\
                        AI_data[['Course Names','Course Descriptions',"Chapter Links"]]],sort=False,ignore_index=True)



In [0]:
# business course 
s_bs_data=pd.read_csv("section_topics_ba.csv",index_col=0)

#technical courses
s_coursera_tech_data=pd.read_csv("section_topics.csv",index_col=0)
#
s_sql_data=pd.read_csv("section_topics_sql.csv",index_col=0)
s_big_data=pd.read_csv('section_topic_bigdata.csv',index_col=0)
s_java_data=pd.read_csv("section_topics_java.csv",index_col=0)
s_la_data=pd.read_csv('section_topics_la.csv',index_col=0)
s_nlp_data=pd.read_csv("section_topics_nlp.csv",index_col=0)
s_python_data=pd.read_csv("section_topics_python.csv",index_col=0)
s_scala_data=pd.read_csv("section_topics_scala.csv",index_col=0)
s_stats_data=pd.read_csv("section_topics_stats.csv",index_col=0)

#AI
s_AI_data=pd.read_csv("section_topics_ai.csv")



In [0]:
df_all_section=pd.concat([s_bs_data[['Section Names','Section Descriptions','Chapter Links']],\
                        s_coursera_tech_data[['Section Names','Section Descriptions','Chapter Links']],\
                        s_sql_data[['Section Names','Section Descriptions','Chapter Links']],\
                        s_big_data[['Section Names','Section Descriptions','Chapter Links']],\
                        s_java_data[['Section Names','Section Descriptions','Chapter Links']],\
                        s_la_data[['Section Names','Section Descriptions','Chapter Links']],\
                        s_nlp_data[['Section Names','Section Descriptions','Chapter Links']],\
                        s_python_data[['Section Names','Section Descriptions','Chapter Links']],\
                        s_scala_data[['Section Names','Section Descriptions','Chapter Links']],\
                        s_stats_data[['Section Names','Section Descriptions','Chapter Links']],\
                        s_AI_data[['Section Names','Section Descriptions','Chapter Links']]],sort=False,ignore_index=True)




## Build Information Retrieval (IR)
The core of recommer system is to build an information retrieval function. Information retrieval (IR) is finding material (usually documents) of an unstructured nature (usually text) that satisfies an information need from within large collections. The core is to use Whoosh, which is a library of classes and functions for indexing text and then searching the index. It allows you to develop custom search engines for your content. Here, the process is to filter the stop word, lower case all the word, stemming the word, and then tokenizer each word to be a token. Then, we need to index each token, and store the file locally. In this part, we need to index the online course description, detailed section of each course, UofT courses, and courses designed by us

In [0]:
Analyzer = RegexTokenizer() | LowercaseFilter() | IntraWordFilter() | StopFilter() | StemFilter()


#%%
# import data into pandas df and create index schema

online_course = pd.DataFrame(df_all_course,copy=True)
schema = Schema(path = ID(stored=True),
                link = ID(stored=True),
                title = TEXT (stored = True),
                text = TEXT(analyzer = Analyzer))

#%%
# create and populate index
def populate_index(dirname, dataframe, schema):
    # Checks for existing index path and creates one if not present
    if not os.path.exists(dirname):
        os.mkdir(dirname)
    print("Creating the Index")
    ix = index.create_in(dirname, schema)
    with ix.writer() as writer:
        # Imports stories from pandas df
        print("Populating the Index")
        for i in dataframe.index:
            add_stories(i, dataframe, writer)
    
def add_stories(i, dataframe, writer):   
    writer.update_document(path= str(dataframe.index[i]),
                           link=str(dataframe.loc[i, "Chapter Links"]),
                           title = str(dataframe.loc[i, "Course Names"]),
                           text = str(dataframe.loc[i, "Course Descriptions"]))
                           


              
## build a index file 
populate_index("online_course_Index", online_course, schema)




Creating the Index
Populating the Index


In [0]:

#%%
# import data into pandas df and create index schema


schema = Schema(path = ID(stored=True),
                link = ID(stored=True),
                title = TEXT (stored = True),
                text = TEXT(analyzer = Analyzer))

#%%
# create and populate index
def populate_index_1(dirname, dataframe, schema):
    # Checks for existing index path and creates one if not present
    if not os.path.exists(dirname):
        os.mkdir(dirname)
    print("Creating the Index")
    ix = index.create_in(dirname, schema)
    with ix.writer() as writer:
        # Imports stories from pandas df
        print("Populating the Index")
        for i in dataframe.index:
            add_stories_1(i, dataframe, writer)
    
def add_stories_1(i, dataframe, writer):   
    writer.update_document(path= str(dataframe.index[i]),
                           link=str(dataframe.loc[i, "Chapter Links"]),
                           title = str(dataframe.loc[i, "Section Names"]),
                           text = str(dataframe.loc[i, "Section Descriptions"]))
                           
              
## build a index file 
populate_index_1("online_section_Index", df_all_section, schema)




Creating the Index
Populating the Index


In [0]:
schema = Schema(path = ID(stored=True),
                title = TEXT (stored = True),
                text = TEXT(analyzer = Analyzer))

#%%
# create and populate index
def populate_index_2(dirname, dataframe, schema):
    # Checks for existing index path and creates one if not present
    if not os.path.exists(dirname):
        os.mkdir(dirname)
    print("Creating the Index")
    ix = index.create_in(dirname, schema)
    with ix.writer() as writer:
        # Imports stories from pandas df
        print("Populating the Index")
        for i in dataframe.index:
            add_stories_2(i, dataframe, writer)
    
def add_stories_2(i, dataframe, writer): 

    writer.update_document(path= str(dataframe.index[i]),
                           title = str(dataframe.loc[i, "Course Names"]),
                           text = str(dataframe.loc[i, "Course Descriptions"]))
                           
              
## build a index file 
populate_index_2("uoft_course_Index", uoft_data, schema)

populate_index_2("own_course_Index", own_data, schema)



Creating the Index
Populating the Index
Creating the Index
Populating the Index


## Build Recommendation System

Here, we need to build a search function. The function need to find the most related courses to a quary. The quary is obtained from students' demand. The difficulty of this part is to choose the proper scoring method, and searching method. We want our recommendation system can provide the courses containing more useful information, and try to cover more demand of the student. Therefore, we choose to use OrGroup method as search method, and implements the BM25F scoring algorithm. The OrGroup method is to choose the course if any keyword appear in. Therefore, it will help us select many related course.Then, we choose BM25 to find the top ranking courses. BM25 is a bag-of-words retrieval function that ranks a set of documents based on the query terms appearing in each document, regardless of their proximity within the document.

In [0]:
#%%
# creates index searcher
def index_search(dirname, search_fields, search_query):
    ix = index.open_dir(dirname)
    schema = ix.schema
    # Create query parser that looks through designated fields in index

    og = qparser.OrGroup.factory(0.8)
#     group = og
    mp = qparser.MultifieldParser(search_fields, schema,group = og)

    # This is the user query
    q = mp.parse(search_query)
    path_list=[]
    course_list=[]
    link_list=[]
    # open the searcher 
    s= ix.searcher()
    # Actual searcher, prints top 20 hits. chooose the top 20 course.
    results = s.search(q, limit = 30,terms=True)
    for (docnum, result) in enumerate(results):
        #score = results.score(docnum)
        path_list.append(result["path"])
        course_list.append(result["title"])
        link_list.append(result["link"])
    
    
    return path_list,course_list,link_list

def index_search_1(dirname, search_fields, search_query):
    ix = index.open_dir(dirname)
    schema = ix.schema
    # Create query parser that looks through designated fields in index

    og = qparser.OrGroup.factory(0.8)
#     group = og
    mp = qparser.MultifieldParser(search_fields, schema,group = og)

    # This is the user query
    q = mp.parse(search_query)
    path_list=[]
    course_list=[]
    # open the searcher 
    s= ix.searcher()
    # Actual searcher, prints top 20 hits. chooose the top 20 course.
    results = s.search(q, limit = 5,terms=True)
    for (docnum, result) in enumerate(results):
        #score = results.score(docnum)
        path_list.append(result["path"])
        course_list.append(result["title"])  
    
    return path_list,course_list
def present_result(dirname, search_fields, search_query,df_original):
    path,course,link=index_search(dirname, search_fields, search_query)
    df=pd.DataFrame(list(zip(path,course,link)),columns=["path","course","link"])
    df=df.drop_duplicates(subset=['course'])
    index=df['path'].tolist()
    df_=df_original.iloc[index]
    return df_
def present_result_1(dirname, search_fields, search_query,df_original):
    path,course=index_search_1(dirname, search_fields, search_query)
    df=pd.DataFrame(list(zip(path,course)),columns=["path","course"])
    df=df.drop_duplicates(subset=['course'])
    index=df['path'].tolist()
    df_=df_original.iloc[index]
    return df_

In [0]:
survey_list=[]

### test for search topic related course
Here, we do a little test for our recommer system. First, the query is "natural language processing and python". And then, it gives us the suggested couresd name, course description and website link for us to search. From the result, we can see the recommer system works. The course give back all related to the natural language processing.

In [0]:
df=present_result("online_course_Index", ['text'], u"natural language processing and python",df_all_course)
df

,Course Names,Course Descriptions,Chapter Links
1158,Introduction to Natural Language Processing in R,"As with any fundamentals course, Introduction ...",NaN
68,NATURAL LANGUAGE PROCESSING,This course focuses on the principles and tech...,NaN
57,ARTIFICIAL INTELLIGENCE FUNDAMENTALS,This course introduces students to the field o...,NaN
413,Deep Learning Specialization,"If you want to break into AI, this Specializat...",https://www.coursera.org/learn/neural-networks...
171,Python for Everybody Specialization,This Specialization builds on the success of t...,https://www.coursera.org/learn/python?speciali...


## Questionnaire for student 

We list five question to ask for some information of the user, and help them to find the neede courses. First, the recommender system want to know what kind of job uses look for. Then, what programming language the job requires, or the user like to use. Next, what skills the user want. After that, management skills are also very important for a job. At last, the users can add any skills and knowledge they want to learn. In each question, users can select to a single choice, or multiply choice. The recommender system will help them to find all related high ranking courses.

In [0]:
##empty first
survey_list=[]

#### Question 1: What kind of job do you want to find? (eg. big data, machine learning, natural language processing, ex..)


In [0]:
keyword=input()
survey_list.append(keyword)

big data


#### Question 2: what kind of programming language do you want to learn? (eg. python, java, R, SQL, ex..)

In [0]:
keyword=input()
survey_list.append(keyword)

python


#### Question 3: What kind of skills do you need to learn for your future career? (eg. visualization, mathematics, SAS, ...ex )

In [0]:
keyword=input()
survey_list.append(keyword)

visualization


#### Question 4: What kind of management ability do you need to learn for your future career? (eg. decision making, teamwork, presentation,...ex )

In [0]:
keyword=input()
survey_list.append(keyword)

presentation


#### Question 5: What else knowledge do you want to learn to improve yourself? (eg. you can type nothing or linear optimization ex.. )

In [0]:
keyword=input()
survey_list.append(keyword)

leadership


In [0]:
survey_list=survey_list
survey_list=str(survey_list).lower()

In [0]:
survey_list

"['big data', 'python', 'visualization', 'presentation', 'leadership']"

## Recommender System Results  (Recommended courses)

### From Online Courses 

In [0]:
df=present_result("online_course_Index", ['text'], survey_list,df_all_course)
df


,Course Names,Course Descriptions,Chapter Links
1150,Interactive Data Visualization with Bokeh,Bokeh is an interactive data visualization lib...,NaN
135,Data Visualization with Tableau Specialization,In 2020 the world will generate 50 times the a...,https://www.coursera.org/learn/data-visualizat...
344,Applied Data Science Specialization,This is an action-packed specialization is for...,https://www.coursera.org/learn/python-for-appl...
289,Data Science at Scale Specialization,"Learn scalable data management, evaluate big d...",https://www.coursera.org/learn/data-manipulation


### From detailed section of online courses
This part, we look for useful course from each section of each online course. This can help student release learning load, and save more time to learn

In [0]:
df=present_result("online_section_Index", ['text'], survey_list,df_all_section)
df

,Section Names,Section Descriptions,Chapter Links
946,Leadership’s Role in Analytics with Florian Ze...,Why the data explosion in marketing has made a...,https://www.coursera.org/learn/leadership-mark...
861,Module 2: ​Big ​Data ​Tools ​Overview,Learn what are the key big data tools on Googl...,https://www.coursera.org/learn/gcp-exploring-p...
2731,Storing and Exporting Data,Learn what are the key big data tools on Googl...,https://www.coursera.org/learn/gcp-creating-bi...
4954,Module 1: Introduction ​to ​Data ​on Google ​C...,Learn what are the key big data tools on Googl...,https://www.coursera.org/learn/gcp-exploring-p...
3238,Summarising Data,Charts are one of the most common ways to pres...,https://www.coursera.org/learn/excel-intermedi...
162,Data Visualization,This week we are going to focus on data visual...,https://www.coursera.org/learn/advanced-excel?...
2689,Delivering a presentation,This week we are going to focus on data visual...,https://www.coursera.org/learn/powerpoint-pres...
5204,Advanced Scenario Analysis,This week we are going to focus on data visual...,https://www.coursera.org/learn/advanced-excel?...
1151,R Markdown and Leaflet,"In this module, we'll dive into the world of c...",https://www.coursera.org/learn/data-products
7496,R Packages,"In this module, we'll dive into the world of c...",https://www.coursera.org/learn/data-products?s...


### From UofT elective courses at Emphasis in Analytic
we can see MIE 1624 is a very useful course to learn

In [0]:
df=present_result_1("uoft_course_Index", ['text'], survey_list,uoft_data)
df

,Course Names,Course Descriptions
1,MIE 1624H Introduction to Data Science and Ana...,The objective of the course is to learn analyt...
31,MIE1628H: Big Data Science,This course is designed to provide students wi...
11,CHE1147H: Data Mining in Engineering,An exceptional ability to deal with data is th...
26,MIE 1413H: Statistical Models in Empirical Res...,This course covers various statistical models ...
14,CIV 1506H: Freight Transportation and ITS Appl...,Efficient movement of freight is crucial for n...


### From Our designed program courses

In [0]:

df=present_result_1("own_course_Index", ['text'], survey_list,own_data)
df

,Course Names,Course Descriptions
3,Big Data Science,You will gain an understanding of what insight...
15,Data Warehousing for Business Intelligence,This course covers data architecture skills. S...
9,Introduction to Cloud Computing in Big Data,This course covers the basic ideas in cloud co...
18,Leadership and Management,This course is designed to let students to get...
5,Excel for Business,This course is desgined to wo kinds of learner...
